## 

In [1]:
import pandas as pd
import numpy as np

In [6]:
cards = pd.read_json('scryfall-oracle-cards.json')

In [7]:
cards.columns

Index(['all_parts', 'arena_id', 'artist', 'artist_ids', 'booster',
       'border_color', 'card_back_id', 'card_faces', 'cmc', 'collector_number',
       'color_identity', 'color_indicator', 'colors', 'digital', 'edhrec_rank',
       'flavor_text', 'foil', 'frame', 'frame_effects', 'full_art', 'games',
       'hand_modifier', 'highres_image', 'id', 'illustration_id', 'image_uris',
       'lang', 'layout', 'legalities', 'life_modifier', 'loyalty', 'mana_cost',
       'mtgo_foil_id', 'mtgo_id', 'multiverse_ids', 'name', 'nonfoil',
       'object', 'oracle_id', 'oracle_text', 'oversized', 'power', 'preview',
       'prints_search_uri', 'promo', 'promo_types', 'rarity', 'related_uris',
       'released_at', 'reprint', 'reserved', 'rulings_uri', 'scryfall_set_uri',
       'scryfall_uri', 'set', 'set_name', 'set_search_uri', 'set_type',
       'set_uri', 'story_spotlight', 'tcgplayer_id', 'textless', 'toughness',
       'type_line', 'uri', 'variation', 'variation_of', 'watermark'],
      dty

This changes the color identity from a list into a string:

In [54]:
cards['color_identity'] = cards['color_identity'].apply(lambda x: "".join(x))

This is all cards whose color identity is exactly two colors, and type line contains 'creature', but not 'token'.

In [60]:
two_color = cards[(cards['color_identity'].apply(lambda x: len(x)) == 2)&(cards['type_line'].str.contains('Creature',na=False))&~(cards['type_line'].str.contains('Token',na=False))].copy()

This removes everything before and including the dash. The Sen Triplets, for example, would go from:

`Legendary Artifact Creature - Human Wizard`

to both `Human` and `Wizard`, removing `Legendary Artifact Creature`


In [62]:
two_color['type_line'] = two_color['type_line'].str.split(" — ").str[1]

This line:
- splits up all of the remaining types
- puts them into separate columns
- 'stacks' them into a single column, retaining the ID
-- each entry with more than one type has that many rows
- joins on the base data to retain color
- returns types and colors


In [92]:
type_color = two_color['type_line'].str.split(expand=True).stack().reset_index().merge(two_color,left_on='level_0',right_on=two_color.index).iloc[:,[2,13]]

This changes the column head to a more readable type:

In [97]:
type_color.rename(columns={0:'type'},inplace=True)

Groups by color and type, giving the count of the entries as the intersection of the row/column. There are still some extraneous entries at this point (e.g. 'sorcery' and '\\'). Those are manually deleted.

In [109]:
type_color.groupby(['type','color_identity']).size().unstack().fillna(0).to_csv('type_color_counts.csv')